# Класс tf.Module для определения моделей

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np

2025-08-08 21:01:02.654299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754676062.686977   11973 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754676062.694748   11973 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754676062.711603   11973 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754676062.711645   11973 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754676062.711647   11973 computation_placer.cc:177] computation placer alr

In [2]:
class DenseNN(tf.Module):
    def __init__(self, outputs):
        super().__init__()
        self.outputs = outputs
        self.fl_init = False

    def __call__(self, x):
        if not self.fl_init:
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name="w")
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name='b')

            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b)

            self.ft_init = True

        y = x @ self.w + self.b
        return y    

In [3]:
model = DenseNN(1)

In [4]:
print(model(tf.constant([[1.0, 2.0]])))

tf.Tensor([[0.175012]], shape=(1, 1), dtype=float32)


E0000 00:00:1754676067.376104   11973 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1754676067.377251   11973 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
x_train = tf.random.uniform(minval=0, maxval=10, shape=(100, 2))
y_train = [a + b for a, b in x_train]

In [6]:
x_train[:5].numpy()

array([[6.9736147 , 5.2648745 ],
       [9.438627  , 4.919257  ],
       [8.537796  , 9.513378  ],
       [9.0712185 , 0.83719015],
       [3.5380077 , 5.8911157 ]], dtype=float32)

In [7]:
y_train[:5]

[<tf.Tensor: shape=(), dtype=float32, numpy=12.238489151000977>,
 <tf.Tensor: shape=(), dtype=float32, numpy=14.357884407043457>,
 <tf.Tensor: shape=(), dtype=float32, numpy=18.05117416381836>,
 <tf.Tensor: shape=(), dtype=float32, numpy=9.908409118652344>,
 <tf.Tensor: shape=(), dtype=float32, numpy=9.429122924804688>]

In [17]:
loss = lambda x, y: tf.reduce_mean(tf.square(x - y))
opt = tf.optimizers.Adam(learning_rate=0.01)

In [18]:
EPOCHS = 50
for n in range(EPOCHS):
    for x, y in zip(x_train, y_train):
        x = tf.expand_dims(x, axis=0)
        y = tf.constant(y, shape=(1, 1))

    with tf.GradientTape() as tape:
        f_loss = loss(y, model(x))

    grads = tape.gradient(f_loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables))

print(f_loss.numpy())            

ValueError: Unknown variable: <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>. This optimizer can only be called for the variables it was originally built with. When working with a new set of variables, you should recreate a new optimizer instance.

In [19]:
for x, y in zip(x_train, y_train):
        x = tf.expand_dims(x, axis=0)
        y = tf.constant(y, shape=(1, 1))

In [20]:
x

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[3.2814097, 3.103248 ]], dtype=float32)>

In [21]:
y

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[6.384658]], dtype=float32)>